In [ ]:
import numpy as np
import tensorflow as tf
import re
import time


#For data conversion:

Data should be consisting of a given and response:
ex. Person 1: What do you mean? What are you talking about? How are you going to do this? (train 1)

ex. Person 2: I won't explain myself (response 1)

The approach to feeding a seq2seq model is similar to the bag-of-words model input but the input is only numbered position of the word in the current position that it takes in the sentence this includes any punctuation. The train response is therefore the response to whatever the input was with the same structure. It's important to have this back and forth element as this is how conversations work.

Should always have a placeholder "[1,...,2]" to denote the beginning and end of person's coversation line.

ex input:

How are you? (How = 4, are = 94, you = 123, ? = 230) becomes:

[1, 4 , 94, 123, 230, 2]

Easiest way to store the data can be in a dictionary, but can use other data types if desired for greater speed.


#Other Data Cleaning Ideas:

1) Change/Match abbreviations, so can't gets transformed to cannot etc.
This can depend on what type of text input you're using.

2) List can/should include slang terms that may not be found in traditional dictionary, as this would be used in natural language.

3) If using messaging/texting data, should remove any special characters ($,#,@,^, -, and others) before using in processing.

4) Certain punctuations such as "," don't need to be included, as these don't indicate a change in tone compared to other quotations such as "?" or "!". Special types of punctuations such as "..." denoting silence should be disucssed whether or not they should be included.

5) Proper nouns and names need to have a special case or token that will be carried over into the response and are then re-established when a network response is produced

6) Can also denote the text length of each input and response to later encode hard limits on how long a response should be - may not be necessary depending on if training data is of good quality or not.

In [ ]:
#Tensorflow seq2seq Model Architecture
#we use tensor arrays "tensors"

#creates our tensor input
#need input type, dimension, name of input

#also want hyper parameters:
#learn rate and "keep_probability" rate or dropout rate ~ 20% of neurons are deactivated
def model_inputs():
  input_array = tf.placeholder(tf.int32,[None, None], name= "input")
  target_array = tf.placeholder(tf.int32,[None, None], name= "target")
  lr = tf.placeholder(tf.float32, name="learn_rate")
  keep_prob = tf.placeholder(tf.float32, name="keep_probability")

  return input_array, target_array, lr, keep_prob




In [ ]:
from numpy.matrixlib import test

#rnn_size is input size of tensors
#seq len is the length of list of each batch
#recall encoder is input "phase"

#LSTM Model here:
def encoder_rnn_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
  lstm = tf.contrib.rnn.BLSTMCell(rnn_size)
  lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
  encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
  encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell, cell_bw = encoder_cell, sequence_length = sequence_length, input = rnn_inputs, dtype = tf.float32) #forward and backward cell must match in input size

  return encoder_state

#decoder is model output "phase"
#decoder is using embeddings which is a word that is translated to a vector

def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input,sequence_length, decoding_scope, output_function, keep_prob, batch_size):
  attention_state = tf.zeros([batch_size, 1,decoder_cell.output_size()])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
  training_decoder_function = tf.contrib.seq2seq.attention_encoder_fn_train(encoder_state[0],attention_keys, attention_values, attention_score_function, attention_construct_function, name = "attn_dec_train") #decodes train set
  decoder_output, decoder_final_state, decoder_final_context_state =  tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, training_decoder_function, decoder_embedded_input, sequence_length, scope = decoding_scope)
  decoder_ouput_dropout = tf.nn.dropout(decoder_output, keep_prob)

  return output_function(decoder_output_dropout)


def decode_test_set(encoder_state, decoder_cell, decoder_embeddedings_matrix,sos_id, eos_id, maximum_length, num_words,sequence_length, decoding_scope, output_function, keep_prob, batch_size):
  attention_state = tf.zeros([batch_size, 1,decoder_cell.output_size()])
  attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
  test_decoder_function = tf.contrib.seq2seq.attention_encoder_fn_inference(output_function, encoder_state[0],attention_keys, attention_values, attention_score_function, attention_construct_function, decoder_embeddedings_matrix,sos_id, eos_id, maximum_length, num_words, name = "attn_dec_inf")
  test_predictions, decoder_final_state, decoder_final_context_state =  tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell, test_decoder_function, scope = decoding_scope)

  return test_predictions

#decoder rnn
#LSTM layer
def decoder_rnn(decoder_embedded_input, decoder_embeddedings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
  with tf.variable_scope("decoding") as decoding_scope:
    lstm = tf.contrib.rnn.BLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    weights = tf.truncate_normal_initializer(stddev = 0.001)
    biases = tf.zeros_initializer()
    output_function = lambda x: tf.contrib.layers.fully_connected(x, num_words, None, scope = decoding_scope, weights_initializers = weights, biases_initializer = biases)

    training_predictions = decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size)

    decoding_scope.reuse_variables()
    test_predictions = decode_test_set(encoder_state, decoder_cell, decoder_embeddedings_matrix, word2int["<SOS>"], word2int["<EOS>"], sequence_length-1, num_words, decoding_scope, output_function, keep_prob, batch_size)

    return training_predictions, test_predictions


#seq2seq model
#brain of chatbot
def seq2seq(inputs, target, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
  encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs, answers_num_words + 1, encoder_embedding_size,initializer = tf.random_uniform_initializer(0, 1))
  encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
  preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
  decoder_embedding_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
  decoder_embedded_input = tf.nn.embedding_lookup(decoder_embedding_matrix, preprocessed_targets)
  training_predictions, test_predictions = decoder_rnn(decoder_embedded_input, decoder_embeddedings_matrix, encoder_state, questions_num_words, sequence_length, rnn_size, num_layers, questionswords2int, keep_prob, batch_size)

  return training_predictions, test_predictions


#Parameters:

Learn Rate: 0.001

Decay = 0.9

Epochs = 1000

Rnn Size = 512

Batch Size = 64

Keep Probability = .50